In [30]:
### Path
project_dir = "/Users/skku_aws26/Downloads/skku_project/BraTS2023_Task1_WinnerSolution_Modified"
training_data   = f'{project_dir}/data/ASNR-MICCAI-BraTS2023-GLI-Challenge-TrainingData'
validation_data = f'{project_dir}/data/ASNR-MICCAI-BraTS2023-GLI-Challenge-ValidationData'
gligan_path = f'{project_dir}/src/GliGAN'
output_path = f'{project_dir}/result'

In [41]:
import os
import pandas as pd

In [32]:
### Change to GliGAN/src/train directory
os.chdir(f'{gligan_path}/src/train')
os.listdir('.')

['tumour_main_baseline.py',
 'tumour_main_modified.py',
 'csv_creator_modified.py',
 'label_main.py']

## 1. CSV file

In [37]:
### Create the csv file
###     1) directory 'Checkpoint' will be forced to be created at ../../Checkpoint
!python csv_creator_modified.py \
    --logdir brats2023 \
    --dataset Brats2023 \
    --datadir {training_data} \
    --outputdir {output_path} \
    --t1c_ending 't1ce.nii.gz' \
    --debug True


Directory /Users/skku_aws26/Downloads/skku_project/BraTS2023_Task1_WinnerSolution_Modified/result/brats2023 created
Directory /Users/skku_aws26/Downloads/skku_project/BraTS2023_Task1_WinnerSolution_Modified/result/brats2023/debug created
CSV_PATH: /Users/skku_aws26/Downloads/skku_project/BraTS2023_Task1_WinnerSolution_Modified/result/brats2023/brats2023.csv
Doing case ID: 00045-001
Case id: 00045-001 was skipped (one dimention bigger than 96)
Doing case ID: 00046-000
Doing case ID: 00045-000
Doing case ID: 00051-000
Case id: 00051-000 was skipped (one dimention bigger than 96)
Doing case ID: 00044-000
Doing case ID: 00043-000
Doing case ID: 00032-001
Doing case ID: 00031-000
Case id: 00031-000 was skipped (one dimention bigger than 96)
Doing case ID: 00025-000
Doing case ID: 00019-000
Case id: 00019-000 was skipped (one dimention bigger than 96)
Doing case ID: 00018-000
Doing case ID: 00024-000
Doing case ID: 00030-000
Doing case ID: 00031-001
Case id: 00031-001 was skipped (one diment

In [38]:
### Check CSV file
csv_file = f'{output_path}/brats2023/brats2023.csv'
os.path.exists(csv_file)

True

In [63]:
### Explore the csv file
df = pd.read_csv(csv_file)
data_size = len(df[:])
data_size

37

## GliGAN Training
* Since the training will be iterated for 8 times, define the function

In [75]:
def get_training_cmd(config, training_file, modality):
    cmd = ["python", training_file]
    for parameter, value in config.items():
        cmd.append(f"--{parameter}")
        cmd.append(str(value))
    cmd.append(f"--modality {modality}") #t1, t1ce, t2, flair
    return cmd

### Before Start Your Training,Set the Parameters

In [87]:
batch_size = 2 # number of batches. # default 4
first_num_steps = 10   # final step of the first training.  default=200000
second_num_steps = 100 # final step of the second training. default=2000000

### 'batch_size', 'num_steps', and the size of data in your csv file decide the size of epoch.
ep = first_num_steps/(data_size/batch_size)
print(f'first training epoch: {ep}')
ep = second_num_steps/(data_size/batch_size)
print(f'second training epoch: {ep}')

first training epoch: 0.5405405405405406
second training epoch: 5.405405405405405


In [88]:
epoch_saving = 100 # weights will be saved every epoch under this value, or at the end of the training

In [89]:
second_resume = first_num_steps  
### second trainig starts by loading the weights of the first training
### so it needs to 'resume' from the end of the first training step
### of course if the second training was terminated, fix this value and 'resume' it

In [90]:
first_training_configs = {
    'outputdir': output_path,
    'batch_size': batch_size,
    #'resume_iter': 1, # resume step
                       # if your first training progress was terminated in the middle of the process, you can load the weights from the 'step' of this value
                       # so, set this value only when it is necessary
    'num_steps': first_num_steps, # final step
    'epoch_saving': epoch_saving, # save every this epoch
    'num_workers': 2, # number of cores. default=2
    'logdir': 'brats2023',
    'in_channels': 4,
    'out_channels': 1,
    'optim_lr': 0.0001,
    'reg_weight': 0,
    'noise_type': 'gaussian_extended',
    'not_abs_value_loss': True,
    'use_sigmoid': True,
    'G_n_update': 2,
    'D_n_update': 1,
    'w_loss_recons': 5,
    'dataset': 'Brats_2023',
}

In [91]:
second_training_configs = {
    'outputdir': output_path,
    'batch_size': batch_size,
    'resume_iter': second_resume, # start step
    'num_steps': second_num_steps, # final step
    'epoch_saving': epoch_saving, # save every this epoch
    'num_workers': 2,
    'logdir': 'brats2023',
    'in_channels': 4,
    'out_channels': 1,
    'optim_lr': 0.0001,
    'reg_weight': 0,
    'noise_type': 'gaussian_extended',
    'not_abs_value_loss': True,
    'use_sigmoid': True,
    'G_n_update': 2,
    'D_n_update': 1,
    'w_loss_recons': 100, # updated
    'dataset': 'Brats_2023',
}

### What's different between first and second training?
* Second training starts from the final step of the first training
    * `--resume_iter` = step of the final weight of the first training
* `--w_loss_recons`(Reconstruction Loss Component Weight): 5 -> 100

We just finished the two step gliGAN training of the t1.    
Keep going for t2, t1ce, and flair.

### t1 training

In [96]:
### t1 First Training
cmd = get_training_cmd(first_training_configs, 'tumour_main_modified.py', 't1')
cmd_str = ' '.join(cmd)
#os.system(cmd_str) # run

In [95]:
### t1 Second Training
cmd = get_training_cmd(second_training_configs, 'tumour_main_modified.py', 't1')
cmd_str = ' '.join(cmd)
#os.system(cmd_str) # run

### t1ce training

In [97]:
### t1ce First Training
cmd = get_training_cmd(first_training_configs, 'tumour_main_modified.py', 't1ce')
cmd_str = ' '.join(cmd)
print(cmd_str)
#os.system(cmd_str) # run

python tumour_main_modified.py --outputdir /Users/skku_aws26/Downloads/skku_project/BraTS2023_Task1_WinnerSolution_Modified/result --batch_size 2 --num_steps 10 --epoch_saving 100 --num_workers 2 --logdir brats2023 --in_channels 4 --out_channels 1 --optim_lr 0.0001 --reg_weight 0 --noise_type gaussian_extended --not_abs_value_loss True --use_sigmoid True --G_n_update 2 --D_n_update 1 --w_loss_recons 5 --dataset Brats_2023 --modality t1ce


In [ ]:
### t1ce Second Training
cmd = get_training_cmd(second_training_configs, 'tumour_main_modified.py', 't1ce')
cmd_str = ' '.join(cmd)
print(cmd_str)
#os.system(cmd_str) # run

### t2 training

In [ ]:
### t2 First Training
cmd = get_training_cmd(first_training_configs, 'tumour_main_modified.py', 't2')
cmd_str = ' '.join(cmd)
print(cmd_str)
#os.system(cmd_str) # run

In [ ]:
### t2 Second Training
cmd = get_training_cmd(second_training_configs, 'tumour_main_modified.py', 't2')
cmd_str = ' '.join(cmd)
print(cmd_str)
#os.system(cmd_str) # run

### flair training

In [ ]:
### flair First Training
cmd = get_training_cmd(first_training_configs, 'tumour_main_modified.py', 'flair')
cmd_str = ' '.join(cmd)
print(cmd_str)
#os.system(cmd_str) # run

In [ ]:
### flair Second Training
cmd = get_training_cmd(second_training_configs, 'tumour_main_modified.py', 'flair')
cmd_str = ' '.join(cmd)
print(cmd_str)
#os.system(cmd_str) # run

## Label generator

In [ ]:
!python label_main_modified.py \
    --logdir brats2023 \
    --batch_size 4 \
    --num_worker {num_workers} \
    --in_channels 3 \
    --out_channels 1 \
    --total_iter {first_num_steps} \
    --datadir {training_data} \
    --outputdir {output_path} \



The reason why the iteration following the first training steps is cause the authur set it like that.